In [1]:
import pandas as pd
import os
from time import time
from pytextprocess import *
from dataRetrieval import *
from sklearn.feature_extraction.text import CountVectorizer
from my_latent_dirichlet_allocation import *

In [ ]:
try:
    with open('backup/jokes.pickle', "rb") as f:
        jokes = pickle.load(f)
    f.close()
    with open('backup/preprocessed_jokes.pickle', "rb") as f:
        preprocessed_jokes = pickle.load(f)
    f.close()
except IOError as e:
    print ("I/O error: {0}".format(e))
    preprocesser = Preprocesser(tagger_backup='backup/tagger.pickle')
    jokes, preprocessed_jokes = preprocesser.preprocess()
    with open('backup/jokes.pickle', 'wb') as f:
        pickle.dump(jokes, f)
    f.close()
    with open('backup/preprocessed_jokes.pickle', 'wb') as f:
        pickle.dump(preprocessed_jokes, f)
    f.close()

In [98]:
path = 'data/jester_dataset_2/'
with open(os.path.join(path, 'jester_items.dat')) as f:
    jokes = f.read()
f.close()
html_jokes = jokes.split('</p>')

In [99]:
html_jokes = [re.sub(r'\r\n\n', '', joke) for joke in html_jokes]
html_jokes = [re.sub(r'&quot;', '\"', joke) for joke in html_jokes]
html_jokes = [re.sub('&#039;', "\'", joke) for joke in html_jokes]
html_jokes = [re.sub(r'\n', '', joke) for joke in html_jokes]
html_jokes = [re.sub(r'\r', '', joke) for joke in html_jokes]
html_jokes = [re.sub(r'<p>', '', joke) for joke in html_jokes]
html_jokes = [re.sub(r'<br />', '', joke) for joke in html_jokes]

In [100]:
#extracter = Extracter()
#tokenizer = Tokenizer()
#tagger = Tagger()
#replacer = AntonymReplacer()
#lemmatizer = Lemmatizer()
preprocessed_jokes = []
for joke in html_jokes:
    # Jokes tokenization...
    tokenized_joke = tokenizer.tokenize(joke, remove_stopwords=False)
    # Jokes tokenization done!
    # Jokes POS Tagging...
    tagged_sentences = tagger.tag_sentences(tokenized_joke)
    tagged_joke = tagger.map_tag(tagged_sentences)
    # Jokes POS Tagging done!
    # Jokes negation replacement...
    tagged_joke2 = []
    for sentence in tagged_joke:
        tagged_joke2.append(replacer.replace_negations(sentence))
    # Jokes negation replacement done!
    # Jokes lemmatization...
    lemmatized_joke = lemmatizer.lemmatize_sentences(tagged_joke2)
    # Jokes lemmatization done!
    # Jokes removing stopwords and punctuation...
    normalized_joke = extracter.remove_stopwords(lemmatized_joke)
    # Jokes removing stopwords and punctuation!
    # Jokes transformation from list of tokens into concatenation of tokens...
    preprocessed_joke = ''
    for normalized_sentence in normalized_joke:
        preprocessed_joke = preprocessed_joke +' '+' '.join(normalized_sentence)
    preprocessed_jokes.append(preprocessed_joke.strip())
    # Jokes transformation from list of tokens into concatenation of tokens!
preprocessed_jokes = preprocessed_jokes[:150]

In [104]:
preprocessed_jokes = preprocessed_jokes[:150]
#preprocessed_jokes = [re.sub(r'.', ' ', joke) for joke in preprocessed_jokes]

In [105]:
v = CountVectorizer(token_pattern='[a-zA-Z]{3,}',max_df=0.95, min_df=0.0002,max_features=2000,stop_words='english')
jokes_words = v.fit_transform(preprocessed_jokes)
word_index = v.get_feature_names()

In [106]:
lda = LatentDirichletAllocation(n_topics=20, max_iter=10, learning_method='online',
                                       learning_offset=10., random_state=0, n_jobs=-1)
lda.fit(jokes_words)
jokes_topics = lda.transform(jokes_words)
docs_topics_norm = jokes_topics/jokes_topics.sum(axis=1)[:,None]

In [116]:
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

similarity = 1-pairwise_distances(docs_topics_norm, metric="cosine")
similarity = np.tril(similarity,k=-1)
corre = np.tril(pairwise_distances(docs_topics_norm, metric='correlation'))

In [117]:
c = corre.copy()

index_max = []
for i in range(10):
    index_max.append(unravel_index(c.argmax(), c.shape))
    c[index_max[i][0],index_max[i][1]] = 0

print index_max

for index in index_max:
    print corre[index[0], index[1]]

[(49, 38), (101, 38), (69, 16), (49, 6), (38, 3), (93, 38), (61, 38), (49, 16), (117, 16), (87, 49)]
1.15493556676
1.13612312617
1.12690535413
1.12619195748
1.12479763396
1.12470478971
1.12416542092
1.12304642855
1.12138010134
1.121082644


In [120]:
print html_jokes[69]
print html_jokes[16]

70:Employer to applicant: "In this job we need someone who is responsible."Applicant: "I'm the one you want. On my last job, every time anything went wrong, they said I was responsible."
17:How many men does it take to screw in a light bulb?One. Men will screw anything.


In [113]:
s = similarity.copy()

index_max = []
for i in range(10):
    index_max.append(unravel_index(s.argmax(), s.shape))
    s[index_max[i][0],index_max[i][1]] = 0

print index_max

for index in index_max:
    print similarity[index[0], index[1]]

[(20, 19), (145, 134), (76, 40), (97, 91), (100, 98), (132, 1), (57, 9), (136, 55), (141, 90), (119, 62)]
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.999999992753
0.999999991629
0.999999989243


In [114]:
print html_jokes[19]
print html_jokes[20]

133:The new employee stood before the paper shredder looking confused. "Need some help?" a secretary, walking by, asked. "Yes," he replied, "how does this thing work?" "Simple," she said, taking the fat report from his hand and feeding it into the shredder."Thanks, but where do the copies come out?"
2:This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. "What could they possibly have said to make you move out?""They told me that you were a pedophile."He replied, "That's an awfully big word for a ten year old."


In [ ]:
t0 = time()
retriever = DataRetriever()
data1 = retriever.read_from_csv_file('jester-1-data.csv')
df1 = data1.drop(['#rating'], axis=1).copy()
print 'Shape of df1 : {}'.format(df1.shape)
print 'done in %0.3fs.' % (time() - t0)

In [ ]:
df = df1.replace(to_replace=99, value=np.nan)
print 'Density: {}'.format(np.sum(df.count())/float(nb_user*nb_item))

### Build sparse matrix

In [ ]:
sparse = []
for i, user in df1.iterrows():
    for j, item in user.iteritems():
        if item!=99:
            sparse.append((i,j,item))
sparse_df = pd.DataFrame(sparse, columns=['userid', 'jokeid','rating'])
sparse_df['rating'] += 10

### Need ~334sec

In [ ]:
retriever = DataRetriever()
t0 = time()
B, user_by_rowindex, movie_by_colindex = retriever.build_sparse_matrix(sparse_df, 'userid', 'jokeid', 'rating')
num_users, num_movies = B.shape
print 'done in %0.3fs.' % (time() - t0)
print '{} joke, {} user'.format(num_movies, num_users)
print 'Density: {}'.format(float(B.getnnz()) / (num_users * num_movies))

In [ ]:
from sklearn.decomposition import TruncatedSVD,FactorAnalysis
pca = TruncatedSVD(n_components=71)
pca.fit(B)
X_pca = pca.transform(B)
B_dense = B.todense()
C = pca.components_
print 'Shape of PCA Components : {}'.format(pca.components_.shape)
print 'Shape of X_pca : {}'.format(X_pca.shape)
print 'Shape of B : {}'.format(B_dense.shape)

In [ ]:
print sum(pca.explained_variance_ratio_)
reconstructed_B = X_pca.dot(C)
reconstructed_B = pd.DataFrame(reconstructed_B)

### Data transformation : from excel to dateframe (need ~85.271 sec)

In [ ]:
t0 = time()
path = 'data/xls/'
retriever = DataRetriever()

table1 = retriever.read_from_excel_file('jester-data-1.xls')
table2 = retriever.read_from_excel_file('jester-data-2.xls')
table3 = retriever.read_from_excel_file('jester-data-3.xls')

print 'Shape of table1 : {}'.format(table1.shape)
print 'Shape of table2 : {}'.format(table2.shape)
print 'Shape of table3 : {}'.format(table3.shape)

frames12 = [table1, table2]
df12 = pd.concat(frames12)
print 'Shape of result : {}'.format(df12.shape)

frames13 = [table1, table3]
df13 = pd.concat(frames13)
print 'Shape of result : {}'.format(df13.shape)

frames23 = [table2, table3]
df23 = pd.concat(frames23)
print 'Shape of result : {}'.format(df23.shape)

retriever.store_to_csv(df12, path_to_csv='data/csv/', name='jester-12-data.csv')
retriever.store_to_csv(df13, path_to_csv='data/csv/', name='jester-13-data.csv')
retriever.store_to_csv(df23, path_to_csv='data/csv/', name='jester-23-data.csv')
retriever.store_to_csv(table1, path_to_csv='data/csv/', name='jester-1-data.csv')
retriever.store_to_csv(table2, path_to_csv='data/csv/', name='jester-2-data.csv')
retriever.store_to_csv(table3, path_to_csv='data/csv/', name='jester-3-data.csv')
print 'done in %0.3fs.' % (time() - t0)

### Data loading

In [ ]:
from dataRetrieval import *
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import log_loss
from pyfm import pylibfm
from time import time

#### Need about ~5 sec

In [121]:
path = 'data/jester_dataset_2/'
df = pd.read_csv(os.path.join(path, 'jester_ratings.dat'), sep='\t\t', names=['userid', 'jokeid','rating'])

/home/thebatou/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  ParserWarning)


### Need 230 sec

In [122]:
retriever = DataRetriever()
t0 = time()
B, user_by_rowindex, movie_by_colindex = retriever.build_sparse_matrix(df, 'userid', 'jokeid', 'rating')
num_users, num_movies = B.shape
print 'done in %0.3fs.' % (time() - t0)
print '{} joke, {} user'.format(num_movies, num_users)
print 'Density: {}'.format(float(B.getnnz()) / (num_users * num_movies))

done in 179.618s.
140 joke, 59132 user
Density: 0.212304674288


In [ ]:
from sklearn.decomposition import TruncatedSVD,FactorAnalysis
pca = TruncatedSVD(n_components=71)
pca.fit(B)
X_pca = pca.transform(B)
B_dense = B.todense()
C = pca.components_
print 'Shape of PCA Components : {}'.format(pca.components_.shape)
print 'Shape of X_pca : {}'.format(X_pca.shape)
print 'Shape of B : {}'.format(B_dense.shape)

In [ ]:
plot_embedding(X_pca5, "Principal Components projection")
plot_embedding(X_pca3O, "Principal Components projection")

In [ ]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=2, algorithm='brute', metric='cosine', n_jobs=-1)
neigh.fit(B)


In [ ]:
import numpy as np
from numpy import unravel_index
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

def compute_pearson_correlation(X):
    n,m = X.shape
    pearson_matrix = np.zeros((m,m))
    j = 0
    while j < m:
        for i in range(j+1,m):
            pearson_matrix[j,i] = pearsonr(X[:,j], X[:,i])[0]
        j += 1
    return pearson_matrix

def compute_cosine_similarity(A):
    similarity = A.dot(A.T)
    square_mag = A.diag()
    inv_square_mag = 1 / square_mag
    inv_square_mag[np.isinf(inv_square_mag)] = 0
    inv_mag = np.sqrt(inv_square_mag)
    cosine = similarity * inv_mag
    return cosine.T * inv_mag

In [ ]:
pearson = compute_pearson_correlation(X)
#cosine = compute_cosine_similarity(B)

In [ ]:
p = pearson.copy()

index_max = []
for i in range(5):
    index_max.append(unravel_index(p.argmax(), p.shape))
    p[index_max[i][0],index_max[i][1]] = 0

print index_max

for index in index_max:
    print pearson[index[0], index[1]]

In [ ]:
t0 = time()
data1 = retriever.read_from_csv_file('jester-1-data.csv')
#data2 = retriever.read_from_csv_file('jester-2-data.csv')
#data3 = retriever.read_from_csv_file('jester-3-data.csv')
#data12 = read_from_csv_file('jester-12-data.csv')
#data13 = read_from_csv_file('jester-13-data.csv')
#data23 = read_from_csv_file('jester-23-data.csv')

df1 = data1.drop(['#rating'], axis=1).copy()
print 'Shape of df1 : {}'.format(df1.shape)
#df2 = data2.drop(['#rating'], axis=1).copy()
#print 'Shape of df2 : {}'.format(df2.shape)
#df3 = data3.drop(['#rating'], axis=1).copy()
#print 'Shape of df3 : {}'.format(df3.shape)
#df12 = data12.drop(['#rating'], axis=1).copy()
#print 'Shape of df12 : {}'.format(df12.shape)
#df13 = data13.drop(['#rating'], axis=1).copy()
#print 'Shape of df13 : {}'.format(df13.shape)
#df23 = data23.drop(['#rating'], axis=1).copy()
#print 'Shape of df23 : {}'.format(df23.shape)

print 'done in %0.3fs.' % (time() - t0)

### Need about ~30 sec

In [ ]:
#t0 = time()
#(data1, label1, users1, items1) = build_data(df1)
#(data2, label2, users2, items2) = build_data(df2)
#(data3, label3, users3, items3) = build_data(df3)
#print 'done in %0.3fs.' % (time() - t0)

### Need about ~8 sec

In [ ]:
#t0 = time()
#df1 = pd.DataFrame(data1).join(pd.DataFrame(label1))
#df1.set_axis(1, ['joke_id', 'user_id', 'rating'])
#df2 = pd.DataFrame(data2).join(pd.DataFrame(label2))
#df2.set_axis(1, ['joke_id', 'user_id', 'rating'])
#df3 = pd.DataFrame(data3).join(pd.DataFrame(label3))
#df3.set_axis(1, ['joke_id', 'user_id', 'rating'])
#print 'done in %0.3fs.' % (time() - t0)

In [ ]:
#d1 = df1.copy()
#d1['rating'] = np.floor(d1['rating'])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df1.hist(bins=20, grid=False, column='rating')
plt.xlim(-10, 10)
plt.xlabel('Rating')
plt.ylabel('#Rating')
plt.title('Distribution of ratings')
plt.show()

d1.hist(bins=20, grid=False, column='rating')
plt.xlim(-10, 10)
plt.xlabel('Rating')
plt.ylabel('#Rating')
plt.title('Distribution of ratings')
plt.show()

d1.groupby('user_id')['rating'].count().hist(bins=20, grid=False)
plt.xlabel('Number of ratings')
plt.ylabel('#Users')
plt.title('Number of users with a given rating density')
plt.show()

d1.groupby('user_id')['rating'].mean().hist(bins=20)

In [ ]:
normalized_df1 = pd.DataFrame()
for index, joke in df1.iteritems():
    normalized_df1 = pd.concat([normalized_df1, np.divide(np.subtract(joke,np.mean(joke)),np.std(joke))], axis=1)

In [ ]:
t0 = time()
B = df1.as_matrix()
W = np.ones(df1.shape)
Psi, Phi = incremental_als(B, W, 2)
print loss(Psi, Phi, B, W)
print Psi.dot(Phi)

print 'done in %0.3fs.' % (time() - t0)

#### Need about ~30 sec

In [ ]:
t0 = time()
(train_data, y_train, train_users, train_items) = build_data(df1)
(test_data, y_test, test_users, test_items) = build_data(df2)
print 'done in %0.3fs.' % (time() - t0)

#### Need about ~30 sec

In [ ]:
t0 = time()
v = DictVectorizer()
X_train = v.fit_transform(train_data) # X_train.shape is the number of tuple (user_i, joke_j, rating_ij != 99)
X_test = v.transform(test_data) # X_test.shape is the number of tuple (user_i, joke_j, rating_ij != 99)
print 'done in %0.3fs.' % (time() - t0)

In [ ]:
y_train = 10 - y_train
y_test = 10 - y_test

#### Need about ~16 min for 10 iterations

In [ ]:
t0 = time()
fm = pylibfm.FM(num_factors=10, num_iter=5, verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")
fm.fit(X_train,y_train)
print 'done in %0.3fs.' % (time() - t0)